In [0]:
# Here the code will be for train a large corpus and embedding with Doc2vec 


In [0]:
from collections import defaultdict
from gensim.models import Word2Vec
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import utils
import csv
from tqdm import tqdm
import multiprocessing
import nltk
import random
from gensim.test.test_doc2vec import ConcatenatedDoc2Vec
from sklearn import preprocessing
from nltk.stem.isri import ISRIStemmer

In [0]:
######## Train Model doc2Vec #########
'''
we will pass the train_document file that contain sentences with their Tags 
and return the trained model
...... very sweet ....

'''
def train_model_doc2vec(train_documents):
    cores = multiprocessing.cpu_count()
    model_dbow = Doc2Vec(dm=1, vector_size=300, negative=5, hs=0, min_count=2, sample=0, workers=cores, alpha=0.025,
                         min_alpha=0.001)
    model_dbow.build_vocab([x for x in tqdm(train_documents)])
    train_documents = utils.shuffle(train_documents)
    model_dbow.train(train_documents, total_examples=len(train_documents), epochs=30)
    return model_dbow


In [0]:
############ Train model Dmm ###################
def train_model_Dmm(train_tagged):
    model_dmm = Doc2Vec(dm=1, dm_mean=1, vector_size=300, window=10, negative=5, min_count=1, workers=5, alpha=0.065,
                        min_alpha=0.065)
    model_dmm.build_vocab([x for x in tqdm(train_tagged)])
    for epoch in range(30):
        model_dmm.train(utils.shuffle([x for x in tqdm(train_tagged)]), total_examples=len(train_tagged),epochs=1)
        model_dmm.alpha -= 0.002
        model_dmm.min_alpha = model_dmm.alpha
    return  model_dmm

In [0]:
# we can mix tow model  ConcatenatedDoc2Vec in  gensim
def concatenated_doc2vec_with_Dmm(model_dbow , model_dmm):
    new_model = ConcatenatedDoc2Vec([model_dbow, model_dmm])
    return new_model